<a href="https://colab.research.google.com/github/mourad-ali0001/IPD/blob/main/Final_year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
!pip install dash
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from catboost import CatBoostClassifier
import numpy as np
import pandas as pd

In [ ]:
df_1920 = pd.read_csv("/content/19,20.csv")
df_2021 = pd.read_csv("/content/20,21.csv")
df_2122 = pd.read_csv("/content/21,22.csv")
df_2223 = pd.read_csv("/content/22,23.csv")
df_2324 = pd.read_csv("/content/23,24.csv")
df_2425 = pd.read_csv("/content/24,25.csv")

df_1920["Season"] = "2019/20"
df_2021["Season"] = "2020/21"
df_2122["Season"] = "2021/22"
df_2223["Season"] = "2022/23"
df_2324["Season"] = "2023/24"
df_2425["Season"] = "2024/25"

data = pd.concat(
    [df_1920, df_2021, df_2122, df_2223, df_2324, df_2425],
    ignore_index=True
)
data["Date"] = pd.to_datetime(data["Date"], dayfirst=True)

data = data.sort_values("Date").reset_index(drop=True)

In [ ]:
data.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA
0,E0,2019-08-09,20:00,Liverpool,Norwich,4,1,H,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,2019-08-10,12:30,West Ham,Man City,0,5,A,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,2019-08-10,15:00,Bournemouth,Sheffield United,1,1,D,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,2019-08-10,15:00,Burnley,Southampton,3,0,H,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,2019-08-10,15:00,Crystal Palace,Everton,0,0,D,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.tail()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA
2275,E0,2025-05-25,16:00,Ipswich,West Ham,1,3,A,0,1,...,3.25,3.65,2.23,3.35,3.75,2.28,1.75,2.28,2.00,1.97
2276,E0,2025-05-25,16:00,Fulham,Man City,0,2,A,0,1,...,6.24,4.64,1.53,6.60,4.80,1.56,1.61,2.60,2.10,1.89
2277,E0,2025-05-25,16:00,Bournemouth,Leicester,2,0,H,0,0,...,1.38,5.64,7.82,1.38,5.80,9.20,1.52,2.86,1.99,1.98
2278,E0,2025-05-25,16:00,Liverpool,Crystal Palace,1,1,D,0,1,...,1.40,6.31,6.31,1.39,6.40,7.60,1.34,3.85,1.97,2.01
2279,E0,2025-05-25,16:00,Wolves,Brentford,1,1,D,0,1,...,2.98,3.80,2.31,3.05,3.80,2.42,1.54,2.78,1.89,2.10


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Columns: 133 entries, Div to BFECAHA
dtypes: datetime64[ns](1), float64(108), int64(16), object(8)
memory usage: 2.3+ MB


In [ ]:
data.shape

(2280, 133)

In [ ]:
data = data.iloc[:,1:24]
data.head(30)

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,2019-08-09,20:00,Liverpool,Norwich,4,1,H,4,0,H,...,7,5,9,9,11,2,0,2,0,0
1,2019-08-10,12:30,West Ham,Man City,0,5,A,0,1,A,...,3,9,6,13,1,1,2,2,0,0
2,2019-08-10,15:00,Bournemouth,Sheffield United,1,1,D,0,0,D,...,3,3,10,19,3,4,2,1,0,0
3,2019-08-10,15:00,Burnley,Southampton,3,0,H,0,0,D,...,4,3,6,12,2,7,0,0,0,0
4,2019-08-10,15:00,Crystal Palace,Everton,0,0,D,0,0,D,...,2,3,16,14,6,2,2,1,0,1
5,2019-08-10,15:00,Watford,Brighton,0,3,A,0,1,A,...,3,3,15,11,5,2,0,1,0,0
6,2019-08-10,17:30,Tottenham,Aston Villa,3,1,H,0,1,A,...,7,4,13,9,14,0,1,0,0,0
7,2019-08-11,14:00,Leicester,Wolves,0,0,D,0,0,D,...,1,2,3,13,12,3,0,2,0,0
8,2019-08-11,14:00,Newcastle,Arsenal,0,1,A,0,0,D,...,2,2,12,7,5,3,1,3,0,0
9,2019-08-11,16:30,Man United,Chelsea,4,0,H,1,0,H,...,5,7,15,13,3,5,3,4,0,0


In [ ]:
data = data[[
    "Date", "HomeTeam", "AwayTeam",
    "FTHG", "FTAG", "FTR",

    "HS", "AS", "HST", "AST",
    "HC", "AC",

    "HF", "AF",
    "HY", "AY",
    "HR", "AR"
]]

data = data.sort_values("Date")


In [ ]:
data.shape

(2280, 18)

In [ ]:
list(data.columns)

['Date',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'FTR',
 'HS',
 'AS',
 'HST',
 'AST',
 'HC',
 'AC',
 'HF',
 'AF',
 'HY',
 'AY',
 'HR',
 'AR']

In [ ]:
data.isnull().sum()

,0
Date,0
HomeTeam,0
AwayTeam,0
FTHG,0
FTAG,0
FTR,0
HS,0
AS,0
HST,0
AST,0


In [ ]:
data.head(120)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
0,2019-08-09,Liverpool,Norwich,4,1,H,15,12,7,5,11,2,9,9,0,2,0,0
1,2019-08-10,West Ham,Man City,0,5,A,5,14,3,9,1,1,6,13,2,2,0,0
2,2019-08-10,Bournemouth,Sheffield United,1,1,D,13,8,3,3,3,4,10,19,2,1,0,0
3,2019-08-10,Burnley,Southampton,3,0,H,10,11,4,3,2,7,6,12,0,0,0,0
4,2019-08-10,Crystal Palace,Everton,0,0,D,6,10,2,3,6,2,16,14,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2019-11-09,Leicester,Arsenal,2,0,H,19,8,7,1,5,4,10,10,1,1,0,0
112,2019-11-09,Tottenham,Sheffield United,1,1,D,17,14,5,4,4,4,6,9,2,2,0,0
117,2019-11-10,Man United,Brighton,3,1,H,21,6,11,2,5,2,10,14,2,5,0,0
118,2019-11-10,Wolves,Aston Villa,2,1,H,17,13,5,4,6,3,17,8,4,3,0,0


In [ ]:
data.tail(120)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
2160,2025-02-23,Man City,Liverpool,0,2,A,16,8,5,4,7,5,3,10,0,0,0,0
2161,2025-02-25,Brighton,Bournemouth,2,1,H,11,19,4,5,1,9,12,14,2,1,0,0
2162,2025-02-25,Crystal Palace,Aston Villa,4,1,H,19,6,6,2,3,8,19,10,3,1,0,0
2163,2025-02-25,Wolves,Fulham,1,2,A,18,11,5,5,7,5,8,15,1,3,0,0
2164,2025-02-25,Chelsea,Southampton,4,0,H,19,7,10,2,4,2,6,9,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,2025-05-25,Nott'm Forest,Chelsea,0,1,A,10,6,2,2,7,4,10,11,2,2,0,0
2271,2025-05-25,Southampton,Arsenal,1,2,A,7,23,2,8,5,8,7,8,0,0,0,0
2270,2025-05-25,Newcastle,Everton,0,1,A,17,14,6,6,12,3,12,9,1,4,0,0
2273,2025-05-25,Man United,Aston Villa,2,0,H,25,6,10,1,4,3,10,10,2,2,0,1


In [ ]:
data["FTR"].unique()

array(['H', 'A', 'D'], dtype=object)

In [ ]:
data["FTR"]= data['FTR'].map({"H": 2, "D": 1, "A": 0})

In [ ]:
data["FTR"].unique()

array([2, 0, 1])

In [ ]:
data.head(20)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
0,2019-08-09,Liverpool,Norwich,4,1,2,15,12,7,5,11,2,9,9,0,2,0,0
1,2019-08-10,West Ham,Man City,0,5,0,5,14,3,9,1,1,6,13,2,2,0,0
2,2019-08-10,Bournemouth,Sheffield United,1,1,1,13,8,3,3,3,4,10,19,2,1,0,0
3,2019-08-10,Burnley,Southampton,3,0,2,10,11,4,3,2,7,6,12,0,0,0,0
4,2019-08-10,Crystal Palace,Everton,0,0,1,6,10,2,3,6,2,16,14,2,1,0,1
5,2019-08-10,Watford,Brighton,0,3,0,11,5,3,3,5,2,15,11,0,1,0,0
6,2019-08-10,Tottenham,Aston Villa,3,1,2,31,7,7,4,14,0,13,9,1,0,0,0
7,2019-08-11,Leicester,Wolves,0,0,1,15,8,1,2,12,3,3,13,0,2,0,0
8,2019-08-11,Newcastle,Arsenal,0,1,0,9,8,2,2,5,3,12,7,1,3,0,0
9,2019-08-11,Man United,Chelsea,4,0,2,11,18,5,7,3,5,15,13,3,4,0,0


In [ ]:
data.tail(20)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
2260,2025-05-16,Aston Villa,Tottenham,2,0,2,18,3,7,1,9,1,5,12,1,2,0,0
2261,2025-05-16,Chelsea,Man United,1,0,2,11,4,3,1,7,2,10,14,1,6,0,0
2262,2025-05-18,Everton,Southampton,2,0,2,13,6,5,2,4,5,6,10,1,1,0,0
2263,2025-05-18,West Ham,Nott'm Forest,1,2,0,16,10,5,5,8,7,14,7,6,3,0,0
2264,2025-05-18,Brentford,Fulham,2,3,0,11,16,6,7,5,3,12,8,4,4,0,0
2265,2025-05-18,Leicester,Ipswich,2,0,2,9,20,3,2,2,2,15,12,1,0,0,0
2266,2025-05-18,Arsenal,Newcastle,1,0,2,12,14,4,5,10,7,9,9,3,3,0,0
2267,2025-05-19,Brighton,Liverpool,3,2,2,25,18,12,5,3,3,9,8,1,0,0,0
2268,2025-05-20,Crystal Palace,Wolves,4,2,2,14,12,10,3,0,10,14,8,1,2,0,0
2269,2025-05-20,Man City,Bournemouth,3,1,2,12,8,5,2,3,1,7,13,1,3,1,1


In [ ]:
def team_outcome_from_ftr(row, team):
    ftr = row["FTR"]

    if ftr == 1:
        return "D"
    if ftr == 2:
        return "W" if row["HomeTeam"] == team else "L"
    if ftr == 0:
        return "W" if row["AwayTeam"] == team else "L"


In [ ]:
def weighted_wdl_last5(data, team, date, recency_weights=(1,2,3,4,5)):
    past = data[
        ((data["HomeTeam"] == team) | (data["AwayTeam"] == team)) &
        (data["Date"] < date)
    ].tail(5)

    if len(past) < 5:
        return pd.Series([np.nan, np.nan, np.nan, np.nan])

    rw = np.array(recency_weights, dtype=float)
    rw = rw / rw.sum()

    wW = wD = wL = 0.0
    for i, (_, row) in enumerate(past.iterrows()):
        outcome = team_outcome_from_ftr(row, team)
        if outcome == "W":
            wW += rw[i]
        elif outcome == "D":
            wD += rw[i]
        else:
            wL += rw[i]

    weighted_points = 3*wW + 1*wD
    return pd.Series([wW, wD, wL, weighted_points])


In [ ]:
df_features = data.copy()

df_features[[
    "HomeW5", "HomeD5", "HomeL5", "HomeWeightedPoints5"
]] = df_features.apply(
    lambda row: weighted_wdl_last5(data, row["HomeTeam"], row["Date"]),
    axis=1
)

df_features[[
    "AwayW5", "AwayD5", "AwayL5", "AwayWeightedPoints5"
]] = df_features.apply(
    lambda row: weighted_wdl_last5(data, row["AwayTeam"], row["Date"]),
    axis=1
)

df_features.dropna(inplace=True)


In [ ]:
def rolling_avg_last5(data, team, date, home_col, away_col):
    past = data[
        ((data["HomeTeam"] == team) | (data["AwayTeam"] == team)) &
        (data["Date"] < date)
    ].tail(5)

    if len(past) < 5:
        return np.nan

    vals = []
    for _, r in past.iterrows():
        if r["HomeTeam"] == team:
            vals.append(r[home_col])
        else:
            vals.append(r[away_col])

    return float(np.mean(vals))



df_features["HomeAvgHST5"] = df_features.apply(
    lambda row: rolling_avg_last5(data, row["HomeTeam"], row["Date"], "HST", "AST"),
    axis=1
)

df_features["AwayAvgAST5"] = df_features.apply(
    lambda row: rolling_avg_last5(data, row["AwayTeam"], row["Date"], "HST", "AST"),
    axis=1
)

df_features["HomeAvgHC5"] = df_features.apply(
    lambda row: rolling_avg_last5(data, row["HomeTeam"], row["Date"], "HC", "AC"),
    axis=1
)

df_features["AwayAvgAC5"] = df_features.apply(
    lambda row: rolling_avg_last5(data, row["AwayTeam"], row["Date"], "HC", "AC"),
    axis=1
)

df_features["HomeAvgHF5"] = df_features.apply(
    lambda row: rolling_avg_last5(data, row["HomeTeam"], row["Date"], "HF", "AF"),
    axis=1
)

df_features["AwayAvgAF5"] = df_features.apply(
    lambda row: rolling_avg_last5(data, row["AwayTeam"], row["Date"], "HF", "AF"),
    axis=1
)

df_features["HomeAvgHY5"] = df_features.apply(
    lambda row: rolling_avg_last5(data, row["HomeTeam"], row["Date"], "HY", "AY"),
    axis=1
)

df_features["AwayAvgAY5"] = df_features.apply(
    lambda row: rolling_avg_last5(data, row["AwayTeam"], row["Date"], "HY", "AY"),
    axis=1
)

df_features["HomeAvgHR5"] = df_features.apply(
    lambda row: rolling_avg_last5(data, row["HomeTeam"], row["Date"], "HR", "AR"),
    axis=1
)

df_features["AwayAvgAR5"] = df_features.apply(
    lambda row: rolling_avg_last5(data, row["AwayTeam"], row["Date"], "HR", "AR"),
    axis=1
)

df_features.dropna(inplace=True)

In [ ]:
df_features.head(50)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,...,HomeAvgHST5,AwayAvgAST5,HomeAvgHC5,AwayAvgAC5,HomeAvgHF5,AwayAvgAF5,HomeAvgHY5,AwayAvgAY5,HomeAvgHR5,AwayAvgAR5
50,2019-09-20,Southampton,Bournemouth,1,3,0,25,6,6,3,...,3.8,4.4,5.0,4.8,11.2,9.2,1.4,1.6,0.2,0.0
54,2019-09-21,Leicester,Tottenham,2,1,2,16,11,7,4,...,2.8,5.0,6.8,6.4,9.2,13.0,0.6,2.4,0.0,0.0
55,2019-09-21,Burnley,Norwich,2,0,2,13,11,6,2,...,3.2,4.8,4.8,3.0,9.0,9.0,1.0,1.6,0.0,0.0
53,2019-09-21,Everton,Sheffield United,0,2,0,16,2,3,1,...,3.4,3.0,5.2,7.0,13.8,13.4,2.2,1.4,0.2,0.2
51,2019-09-21,Man City,Watford,8,0,2,28,5,11,4,...,7.6,3.6,8.6,6.4,11.4,12.6,1.6,2.0,0.0,0.0
52,2019-09-21,Newcastle,Brighton,0,0,1,11,16,4,3,...,2.8,3.4,4.6,4.4,7.4,10.0,1.6,0.6,0.0,0.2
56,2019-09-22,Crystal Palace,Wolves,1,1,1,13,14,4,4,...,4.0,3.0,5.4,5.0,14.2,11.8,2.2,1.8,0.0,0.2
57,2019-09-22,West Ham,Man United,2,0,2,8,9,6,3,...,5.0,4.6,5.2,4.6,11.0,10.4,1.6,2.0,0.2,0.0
58,2019-09-22,Arsenal,Aston Villa,3,2,2,21,14,6,9,...,5.2,4.2,5.8,2.8,8.4,11.6,2.4,1.6,0.0,0.2
59,2019-09-22,Chelsea,Liverpool,1,2,0,13,6,2,3,...,6.2,6.8,5.0,8.0,9.6,8.8,1.6,0.4,0.0,0.0


In [ ]:
np.isclose(
    df_features["HomeW5"] + df_features["HomeD5"] + df_features["HomeL5"],
    1.0,
    atol=1e-6
).all()


np.True_

In [ ]:
np.isclose(
    df_features["AwayW5"] + df_features["AwayD5"] + df_features["AwayL5"],
    1.0,
    atol=1e-6
).all()


np.True_

In [ ]:
feature_cols = [
    "HomeW5", "HomeD5", "HomeL5", "HomeWeightedPoints5",
    "AwayW5", "AwayD5", "AwayL5", "AwayWeightedPoints5",
    "HomeAvgHST5", "AwayAvgAST5",
    "HomeAvgHC5", "AwayAvgAC5",
    "HomeAvgHF5", "AwayAvgAF5",
    "HomeAvgHY5", "AwayAvgAY5",
    "HomeAvgHR5", "AwayAvgAR5"
]

X = df_features[feature_cols]
y = df_features["FTR"]


In [ ]:
df_features[feature_cols].describe()

,HomeW5,HomeD5,HomeL5,HomeWeightedPoints5,AwayW5,AwayD5,AwayL5,AwayWeightedPoints5,HomeAvgHST5,AwayAvgAST5,HomeAvgHC5,AwayAvgAC5,HomeAvgHF5,AwayAvgAF5,HomeAvgHY5,AwayAvgAY5,HomeAvgHR5,AwayAvgAR5
count,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000,2196.000000
mean,0.377231,0.233151,0.389617,1.364845,0.396873,0.229630,0.373497,1.420249,4.450273,4.539435,5.176321,5.275410,10.791075,10.733515,1.801548,1.776685,0.058925,0.058743
std,0.267083,0.204940,0.265642,0.745452,0.269657,0.209953,0.265761,0.748343,1.396183,1.412986,1.544618,1.564083,1.822930,1.818314,0.667158,0.665863,0.109201,0.108798
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.400000,1.600000,4.800000,4.600000,0.200000,0.000000,0.000000,0.000000
25%,0.200000,0.000000,0.200000,0.800000,0.200000,0.000000,0.183333,0.800000,3.400000,3.400000,4.200000,4.200000,9.600000,9.600000,1.400000,1.400000,0.000000,0.000000
50%,0.333333,0.200000,0.400000,1.333333,0.333333,0.200000,0.333333,1.400000,4.400000,4.400000,5.000000,5.200000,10.800000,10.800000,1.800000,1.800000,0.000000,0.000000
75%,0.533333,0.333333,0.600000,1.866667,0.600000,0.333333,0.600000,2.000000,5.400000,5.400000,6.000000,6.200000,12.000000,12.000000,2.200000,2.200000,0.200000,0.200000
max,1.000000,1.000000,1.000000,3.000000,1.000000,0.933333,1.000000,3.000000,10.400000,10.800000,12.000000,12.200000,15.800000,16.800000,4.400000,4.600000,0.600000,0.600000


In [ ]:
counts = df_features["FTR"].value_counts().sort_index()
perc = (df_features["FTR"].value_counts(normalize=True).sort_index() * 100).round(2)

balance = pd.DataFrame({"Count": counts, "Percent": perc})
balance.index = balance.index.map({0: "Away Win", 1: "Draw", 2: "Home Win"})
print(balance)


          Count  Percent
FTR                     
Away Win    737    33.56
Draw        499    22.72
Home Win    960    43.72


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False, random_state=22
)

models = {
    "GradientBoosting": GradientBoostingClassifier(),

    "CatBoost": CatBoostClassifier(
        iterations=600,
        learning_rate=0.05,
        depth=6,
        loss_function="MultiClass",
        class_weights={2: 1.0, 1: 1.8, 0: 1.0},
        verbose=False
    ),

    "RandomForest": RandomForestClassifier(
        n_estimators=400,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        n_jobs=-1
    ),

    "DecisionTree": DecisionTreeClassifier(),

    "NeuralNetwork_MLP": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", MLPClassifier(
            hidden_layer_sizes=(64, 32),
            activation="relu",
            solver="adam",
            max_iter=1500,
            early_stopping=True
        ))
    ]),

    "SVM_RBF": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf", C=2.0, gamma="scale"))
    ]),

    "KNN": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=35, weights="distance"))
    ])
}

try:
    from xgboost import XGBClassifier
    models["XGBoost"] = XGBClassifier(
        n_estimators=600,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="multi:softmax",
        num_class=3,
        eval_metric="mlogloss"
    )
except Exception:
    pass

tscv = TimeSeriesSplit(n_splits=5)

cv_rows = []
for name, model in models.items():
    accs, f1s = [], []
    for tr_idx, val_idx in tscv.split(X_train):
        X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        pred_val = model.predict(X_val)

        accs.append(accuracy_score(y_val, pred_val))
        f1s.append(f1_score(y_val, pred_val, average="macro"))

    cv_rows.append({
        "Model": name,
        "CV_Accuracy": float(np.mean(accs)),
        "CV_MacroF1": float(np.mean(f1s))
    })

cv_df = pd.DataFrame(cv_rows).sort_values("CV_MacroF1", ascending=False)
print(cv_df)

test_summary = []
for name, model in models.items():
    model.fit(X_train, y_train)
    pred_test = model.predict(X_test)

    acc = accuracy_score(y_test, pred_test)
    mf1 = f1_score(y_test, pred_test, average="macro")

    test_summary.append({"Model": name, "Test_Accuracy": acc, "Test_MacroF1": mf1})

    print(name)
    print(acc)
    print(mf1)
    print(classification_report(y_test, pred_test, zero_division=0))

test_df = pd.DataFrame(test_summary).sort_values("Test_MacroF1", ascending=False)
print(test_df)

               Model  CV_Accuracy  CV_MacroF1
0   GradientBoosting     0.465753    0.386788
7            XGBoost     0.449315    0.385456
2       RandomForest     0.484247    0.384441
5            SVM_RBF     0.475342    0.384143
1           CatBoost     0.441781    0.383991
6                KNN     0.476027    0.369898
4  NeuralNetwork_MLP     0.477397    0.359966
3       DecisionTree     0.376712    0.352964
GradientBoosting
0.4772727272727273
0.3712745276714659
              precision    recall  f1-score   support

           0       0.50      0.45      0.47       156
           1       0.16      0.03      0.05       101
           2       0.49      0.75      0.59       183

    accuracy                           0.48       440
   macro avg       0.38      0.41      0.37       440
weighted avg       0.42      0.48      0.42       440

CatBoost
0.4022727272727273
0.34505608810494454
              precision    recall  f1-score   support

           0       0.44      0.35      0.39    

In [ ]:
len(list(tscv.split(X_train)))

5